### Set Up Envioronment

In [ ]:
!python3 -m venv venv
!source venv/bin/activate

In [ ]:
!pip3 install "google-cloud-bigquery>=3.17"
!pip3 install "google-cloud-aiplatform>=1.38"
!pip3 install "pandas>=2.2.0"

In [ ]:
import vertexai
from vertexai.preview import generative_models
from vertexai.preview.generative_models import GenerativeModel
from google.cloud import bigquery
from google.cloud import storage
import pandas as pd
import random
import time
import json

project_id = ""

queued_jobs_bucket_name = f"{project_id}-queued-jobs-test"
in_progress_jobs_bucket_name = f"{project_id}-in-progress-jobs-test"
completed_jobs_bucket_name = f"{project_id}-completed-jobs-test"
failed_jobs_bucket_name = f"{project_id}-failed-jobs-test"


dataset_id = ""

raw_target_table = ""
target_table = ""

raw_source_tables = ["source-uipetmis","source-uispet"]
raw_source_tables_wildcard = 'source*'
source_table = ""

output_table = ""

# Initialise BQ client
client = bigquery.Client(project=project_id)

### Get Source and target Schemas from BigQuery

To reproduce yourself, use the .csvs in this directory and upload to BigQuery in your own Project. Do not use the provided spreadsheets directly as I have done some (minimal) pre-processing on the spreadsheets

In [ ]:
def add_unique_ref_and_create_new_table(project_id, dataset_id, raw_table, new_table, new_col):
    """Adds 'Source_Unique_Ref' column if missing, then creates a new BigQuery table.

    Args:
        project_id: BigQuery project ID.
        dataset_id: BigQuery dataset ID.
        raw_tables: Dict containing raw table names
        new_table: Desired name of finalised table
        new_col: the name of the reference column for the table
    """

    raw_query = f"""
        SELECT *  
        FROM `{project_id}.{dataset_id}.{raw_table}`
    """
    raw_df = client.query(raw_query).to_dataframe()
    raw_df[new_col] = range(1, len(raw_df) + 1)  
    new_table_id = f"{project_id}.{dataset_id}.{new_table}"

    job_config = bigquery.LoadJobConfig()  
    job = client.load_table_from_dataframe(raw_df, new_table_id, job_config=job_config)
    job.result()

    return job.result()

In [ ]:
# Source table setup
source_table_ref = client.dataset(dataset_id).table(source_table) 
try:
    client.get_table(source_table_ref)  # Will raise NotFound if the table doesn't exist
    print("Source table '{}' exists.".format(source_table))
except:
    print("Source table '{}' does not exist.".format(source_table))
    print("creating Source table...")
    new_source_col = 'Source_Unique_Ref'
    add_unique_ref_and_create_new_table(project_id, dataset_id, raw_source_tables_wildcard, source_table, new_source_col)

source_query = f"""
    SELECT *  
    FROM `{project_id}.{dataset_id}.{source_table}`
"""
source_df = client.query(source_query).to_dataframe()
print(f"source_df length is {source_df.shape[0]}")


# Target table setup
target_table_ref = client.dataset(dataset_id).table(target_table) 
try:
    client.get_table(target_table_ref)  # Will raise NotFound if the table doesn't exist
    print(f"Target table {target_table} exists.")
except:
    print(f"Target table {target_table} does not exist.")
    print("creating Target table...")
    new_target_col = 'Target_Unique_Ref'   
    add_unique_ref_and_create_new_table(project_id, dataset_id, raw_target_table, target_table, new_target_col)

target_query = f"""
    SELECT *  
    FROM `{project_id}.{dataset_id}.{target_table}`
"""
target_df = client.query(target_query).to_dataframe()
print(f"source_df length is {target_df.shape[0]}")

In [ ]:
source_df.head()

In [ ]:
target_df.head()

### Helper functions to group and format data

In [ ]:
def dataframe_to_string(df):
    """Converts a DataFrame to a string with column names and row values.

    Args:
        df: The pandas DataFrame to convert.

    Returns:
        A string representation of the DataFrame.
    """

    output = f"Column Names: {', '.join(df.columns)}\n"  # Header with column names

    for _, row in df.iterrows():
        row_string = ', '.join(str(value) for value in row)
        output += f"Row: {row_string}\n"

    return output


def parse_function_call(function_call):
    """Parses a FunctionCall object, adds a description, and returns a JSON-compatible dictionary.

    Args:
        function_call: The FunctionCall object to parse.

    Returns:
        A dictionary containing the function name, attributes, and description.
    """

    result = {
        "function_name": function_call.name,
        "attributes": {},
    }
    for key, value in function_call.args.items():
        result["attributes"][key] = value

    return result

def convert_dict_to_list_of_dicts(dict):
    """Converts a dictionary of lists and strings to a list of flat dictionaries.

    Args:
        data: The input dictionary containing lists and strings.

    Returns:
        A list of dictionaries, where each dictionary represents a  
        combination of elements from the input lists.
    """
    
    list_of_attribute_dicts = []
    string_keys = []
    list_keys = []

    for key, value in dict.items() :
        if isinstance(value, str):
            string_keys.append(key)
        else:
            list_keys.append(key)       
    
    for i in range (len(dict[list_keys[0]])):
        new_dict = {}
        
        for key in list_keys:
            new_dict[key] = dict[key][i]
        for key in string_keys:
            new_dict[key] = dict[key]

        list_of_attribute_dicts.append(new_dict)

    return list_of_attribute_dicts

def create_df_from_target_row_df_and_list_of_dicts(list_of_attribute_dicts, test_target_df_row):
    """
    Appends rows to a DataFrame, combining a base row with data from a list of dictionaries.

    Args:
        list_of_dicts: A list of dictionaries, each representing column values.
        test_target_df_row: A DataFrame row containing base columns.

    Returns:
        The modified DataFrame with the newly appended rows.
    """

    df_list = []
     
    for attribute_dict in list_of_attribute_dicts:
        df = test_target_df_row.copy()  # Make a copy 
        for key in attribute_dict.keys(): # Combine the base row with the current dictionary
            df[key] = attribute_dict[key]

        df_list.append(df)

    return pd.concat(df_list)

def merge_dataframes_and_string(target_df_row, source_df_row, confidence_level):
    target_df_row = target_df_row.reset_index(drop=True)
    source_df_row = source_df_row.reset_index(drop=True)
    merged_df = pd.concat([target_df_row, source_df_row], axis=1)
    confidence_df = pd.DataFrame({'Confidence_Levels': [confidence_level]})
    final_df = pd.concat([merged_df, confidence_df], axis=1)
    return final_df


### Prepare the subdivided groups for the source and destination schemas

This is required so we can come in below the maximum token size for Gemini
(32K input, 2K output) https://ai.google.dev/models/gemini#model_variations

In [ ]:
# target_grouping_levels = ['Tranche', 'Level_1', 'Level_2', 'Level_3', 'Level_4']
# target_df_groups = create_df_groups(target_df,target_grouping_levels)

# target_string_groups = []
# for path, target_df_group in target_df_groups.items():
#     target_string = dataframe_to_string(target_df_group)
#     target_string_groups.append(target_string)
    
# print(f"Number of target schema dataframe groupings: {len(target_df_groups)}")
# print(f"Number of target schema string groupings: {len(target_string_groups)}\n")
# print("*************\n")

# source_grouping_levels = ['SchemaName', 'TableName']
# source_df_groups = create_df_groups(source_df, source_grouping_levels)

# source_string_groups = []
# for path, source_group_df in source_df_groups.items():
#     source_group_sting = dataframe_to_string(source_group_df)
#     source_string_groups.append(source_group_sting)
    
# print(f"Number of source schema dataframe groupings: {len(source_df_groups)}")
# print(f"Number of source schema string groupings: {len(source_string_groups)}\n")

# #Further split up the source_df_groups to make sure there is no group larger than maximum_fields_per_request variable. This prevents LLM innacuacies when the number of requested field mappings is too high.
# maximum_fields_per_request = 12

# target_row_number = 42

# chopped_source_df_groups = chop_source_df_groups(source_df_groups, maximum_fields_per_request)
# print(f"Row {target_row_number}: Number of chopped source schema dataframe groupings: {len(chopped_source_df_groups)}")

# chopped_length_counts = {}
# for group_df in chopped_source_df_groups.values():
#     group_length = len(group_df)
#     if group_length in chopped_length_counts:
#         chopped_length_counts[group_length] += 1
#     else:
#         chopped_length_counts[group_length] = 1
# print(f"Row {target_row_number}: Distribution of chopped lengths:")
# for length, count in chopped_length_counts.items():
#     print(f"{count} x groups with length {length}")

# merged_source_df_groups = merge_source_df_groups(chopped_source_df_groups, maximum_fields_per_request)
# print(f"Row {target_row_number}: Number of merged source schema dataframe groupings: {len(merged_source_df_groups)}")

# merged_length_counts = {}
# for group_df in merged_source_df_groups.values():
#     group_length = len(group_df)
#     if group_length in merged_length_counts:
#         merged_length_counts[group_length] += 1
#     else:
#         merged_length_counts[group_length] = 1
# print(f"Row {target_row_number}: Distribution of merged lengths:")
# for length, count in merged_length_counts.items():
#     print(f"{count} x groups with length {length}")

# combined_source_df_groups = merge_small_groups(merged_source_df_groups)
# print(f"Row {target_row_number}: Number of combined source schema dataframe groupings: {len(combined_source_df_groups)}")

# combined_length_counts = {}
# for group_df in combined_source_df_groups.values():
#     group_length = len(group_df)
#     if group_length in combined_length_counts:
#         combined_length_counts[group_length] += 1
#     else:
#         combined_length_counts[group_length] = 1
# print(f"Row {target_row_number}: Distribution of combined lengths:")
# for length, count in combined_length_counts.items():
#     print(f"{count} x groups with length {length}")

# unmapped_source_string_groups = []
# for path, merged_source_df_group in merged_source_df_groups.items():
#     merged_source_string_group = dataframe_to_string(merged_source_df_group)
#     unmapped_source_string_groups.append(merged_source_string_group)
# print(f"Number of merged source schema string groupings: {len(unmapped_source_string_groups)}\n")

# # random_number_in_source_range = random.randint(0, len(unmapped_source_string_groups)-1)
# # print(f"showing randomly chosen source string group {random_number_in_source_range}:")
# # random_source_string_group = unmapped_source_string_groups[random_number_in_source_range]
# # print(random_source_string_group)


### Iterate over all source fields groups (~500), for a single target field

- First we select a single Target schema field

In [ ]:
import re

objectId = "target-row-100-source-groups-0-52"

storage_client = storage.Client()
queued_jobs_bucket = storage_client.bucket(queued_jobs_bucket_name)
failed_jobs_bucket = storage_client.bucket(failed_jobs_bucket_name)
blob = queued_jobs_bucket.blob(objectId)

if not blob.exists():
    print(f"File '{objectId}' not found in bucket '{queued_jobs_bucket_name}'. Job already picked from queue. Container instance completing with 204 message")

pattern = r"^target-row-(\d{1,4})-source-groups-(\d{1,4})-(\d{1,4})$"
match = re.match(pattern, objectId)
if not match:
    msg = "objectId is not in the expected format: ^target-row-(\d{3})-source-groups-(\d{3})-(\d{3})$"
    print(msg)

target_row = int(match.group(1))
source_group_start = int(match.group(2))
source_group_end = int(match.group(3))

print(f"target_row {target_row} source_group_start {source_group_start} source_group_end {source_group_end}")

# Prepare target field
target_df_row = target_df.iloc[[target_row]]
target_string_row = dataframe_to_string(target_df_row)
print("\ntarget_string_row")
print(target_string_row)


contents = blob.download_as_string().decode('utf-8')
unmapped_source_string_groups = []
unmapped_source_string_groups = contents.split("\n\n")  # Split by double newlines

print("\nunmapped_source_string_groups[0]")
print(unmapped_source_string_groups[0])

### Prepare Gemini

- First we create the FunctionDeclaration. This helps us get a more structured and consistent output from the LLM which is helpful for use cases such as this when we are dealing with structured data. See [Function Calling](https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/function-calling)
- Then we prepare the prompt to send to Gemini. The prompt is intentionally very verbose and repetitive, as well as directly refering to the declared function. Further optimisations and improvements could be made if spent tuning the prompt.

In [ ]:
model = GenerativeModel("gemini-pro")

set_source_field_mapping_confidence_levels = generative_models.FunctionDeclaration(
    name="set_source_field_mapping_confidence_levels",
    description="""Sets the mapping confidence values for each source field for a given target field.

Here is a general example to help you understand how to use the set_source_field_mapping_confidences_tool correctly. This is only an example to show the source and target field structures.:

Assuming you had previously decided on the following mapping confidence levels (but it is important that you come up with your own values for mapping condifence level rather than specifically using these values):
a mapping confidence level of 2 for the field with Source_Unique_Ref=158
a mapping confidence level of 1 for the field with Source_Unique_Ref=159
a mapping confidence level of 1 for the field with Source_Unique_Ref=1290
a mapping confidence level of 1 for the field with Source_Unique_Ref=579
a mapping confidence level of 1 for the field with Source_Unique_Ref=638
a mapping confidence level of 1 for the field with Source_Unique_Ref=970
a mapping confidence level of 1 for the field with Source_Unique_Ref=3317
a mapping confidence level of 3 for the field with Source_Unique_Ref=160
a mapping confidence level of 1 for the field with Source_Unique_Ref=1910
a mapping confidence level of 5 for the field with Source_Unique_Ref=2280

Then this function would be used to set the mapping confidence levels for each of the source fields, where your input parameter source_field_mapping_confidences would be:
source_field_mapping_confidences = [
    {'Source_Unique_Ref':158,'mapping_confidence_level':'2'},
    {'Source_Unique_Ref':159,'mapping_confidence_level':'2'},
    {'Source_Unique_Ref':1290,'mapping_confidence_level':'1'},
    {'Source_Unique_Ref':579,'mapping_confidence_level':'1'},
    {'Source_Unique_Ref':638,'mapping_confidence_level':'1'},
    {'Source_Unique_Ref':970,'mapping_confidence_level':'1'},
    {'Source_Unique_Ref':3317,'mapping_confidence_level':'1'},
    {'Source_Unique_Ref':160,'mapping_confidence_level':'3'},
    {'Source_Unique_Ref':1910,'mapping_confidence_level':'1'},
    {'Source_Unique_Ref':2280,'mapping_confidence_level':'5'}
]""",


# Then this function would be used to set the mapping confidence levels for each of the source fields, where your input parameters would be
# 'mapping_confidence_level'=["2", "1", "1", "1", "1", "1", "1", "3", "1", "5"], 'Source_Unique_Ref': [158, 159, 1290, 579, 638, 970, 3317, 160, 1910, 2280]
# And notice that the array index positions for each parameter align with each other to represent the mapping for a particular source field. This is very important.""",
    # parameters={
    #     "type": "object",
    #     "properties": {
    #         "Source_Unique_Ref": {
    #             "type": "array",
    #             "description": "An array containing each of the Source_Unique_Ref values for the set of source fields to set a mapping confidence level for.",
    #             "items" : {
    #                 "type": "integer"
    #             },
    #             "example": [158, 159, 1290, 579, 638, 970, 3317, 160, 1910, 2280]
    #         },
    #         "mapping_confidence_level": {
    #             "type": "array",
    #             "description": "The mapping confidence level for the corresponding source field in the same index in the Source_Unique_Ref parameter. It is very important that the array indexes for mapping_confidence_level align to the Source_Unique_Ref so the mapping confidence levels are aligned to the correct source fields.",
    #             "items" : {
    #                 "type": "string"
    #             },
    #             "example": ["2", "1", "1", "1", "1", "1", "1", "3", "1", "5"]
    #         },
    #     },
    #     "required": ["Source_Unique_Ref", "mapping_confidence_level"]
    # },

    parameters={
        "type": "object",
        "properties": {
            "source_field_mapping_confidences": {
                "type": "array",
                "description": "A List of objects where each object in the list contains the source field's Source_Unique_Ref and the mapping_confidence_level for that source field.",
                "items": {
                    "type": "object",
                    "properties": {
                        "Source_Unique_Ref": {
                            "type": "integer",
                            "description": "The reference ID for the source field."
                        },
                        "mapping_confidence_level": {
                            "type": "string",
                            "enum": ["1", "2", "3", "4", "5"],
                            "description": "The confidence level for the mapping (an integer between 1 and 5)."
                        }
                    },
                    "required": ["Source_Unique_Ref", "mapping_confidence_level"]
                }
            },
        },
        "required": ["source_field_mapping_confidences"],
    },
)

set_source_field_mapping_confidence_levels_tool = generative_models.Tool(
    function_declarations=[set_source_field_mapping_confidence_levels]
)

In [ ]:
single_unmapped_source_string_group = unmapped_source_string_groups[0]
field_count = single_unmapped_source_string_group.count('Row:')

print("************************************")
print(f"{field_count} source fields in group")
print(single_unmapped_source_string_group)
print("mapping to")
print(target_string_row)

prompt = f"""You are Data Engineer working for an insurance company. As part of a data migration project you need to assist with mapping fields in a source data schema fields in a target data schema.
The source and destination schemas are both complex and nested.
You will be shown 1 field in the target schema and multiple fields in the source schema.
The mappings will not be exactly one to one: Instead of providing a one-to-one mapping for a single source schema to a single destiation schema, your job is to provide a mapping confidence level for how well you think each of the fields for the source schemas you see will map to the field for the target schema.

The field from the target schema is described here:
{target_string_row}

The fields taken from the source schema are described here:
{single_unmapped_source_string_group}

Based on your knowledge of the insurance industry, pets, pet insurance, you will provide a mapping confidence level for how well each of the source fields map to the target field.
The confidence level is a number between 1 and 5 where:
1 means there is a very very small chance that the fields could be a match
2 means there is a small chance that the fields colud be a match
3 means there is a medium chance that the fields could be a match
4 means there is a good chance that the fields could be a match
5 means there is a very good chance that the fields could be a match

You should decide on a mapping confidence level for each of the source fields, then set the mapping confidence level for each field using and use the value for Source_Unique_Ref for each source field to reference it with its corresponding mapping confidence level.
Then YOU MUST USE the available function set_source_field_mapping_confidence_levels in the set_source_field_mapping_confidence_levels_tool to set your mappings confidence level for each of the source fields.
YOU MUST USE THIS FUNCTION."""

model_response = model.generate_content(
    prompt,
    generation_config={"temperature": 0},
    tools=[set_source_field_mapping_confidence_levels_tool],
)

if not model_response.candidates[0].content.parts[0].function_call:
    print("did not use fn call! retrying with a more explicit prompt")
    prompt += """
YOU MUST USE THIS FUNCTION."""
    model_response = model.generate_content(
        prompt,
        generation_config={"temperature": 0},
        tools=[set_source_field_mapping_confidence_levels_tool],
    )

function_call_json = parse_function_call(model_response.candidates[0].content.parts[0].function_call)
attributes_dict = function_call_json["attributes"]
print(f"Received mapping response from Gemini: {attributes_dict}")

In [ ]:
from proto.marshal.collections import repeated
from proto.marshal.collections import maps

def recurse_proto_repeated_composite(repeated_object):
    repeated_list = []
    for item in repeated_object:
        if isinstance(item, repeated.RepeatedComposite):
            item = recurse_proto_repeated_composite(item)
            repeated_list.append(item)
        elif isinstance(item, maps.MapComposite):
            item = recurse_proto_marshal_to_dict(item)
            repeated_list.append(item)
        else:
            repeated_list.append(item)

    return repeated_list

def recurse_proto_marshal_to_dict(marshal_object):
    new_dict = {}
    for k, v in marshal_object.items():
      if not v:
        continue
      elif isinstance(v, maps.MapComposite):
          v = recurse_proto_marshal_to_dict(v)
      elif isinstance(v, repeated.RepeatedComposite):
          v = recurse_proto_repeated_composite(v)
      new_dict[k] = v

    return new_dict

In [ ]:
for attribute in attributes_dict['source_field_mapping_confidences']:
    print(recurse_proto_marshal_to_dict(attribute))

In [ ]:
df_list_for_bq_upload = []

for l, mapping_confidence in enumerate(attributes_dict['source_field_mapping_confidences']):
    mapping_confidence_dict = recurse_proto_marshal_to_dict(mapping_confidence)
    Source_Unique_Ref = mapping_confidence_dict['Source_Unique_Ref']
    mapping_confidence_level_int = int(mapping_confidence_dict['mapping_confidence_level'])

    source_df_row = source_df[source_df['Source_Unique_Ref']==Source_Unique_Ref]
    
    mapping_output = merge_dataframes_and_string(target_df_row, source_df_row,mapping_confidence_level_int)
    df_list_for_bq_upload.append(mapping_output)

In [ ]:
df_list_for_bq_upload[0].head()

In [ ]:
print(f"Prepared the mapping as a df ready for upload to bigquery")
print(f"df_list_for_bq_upload contains {len(df_list_for_bq_upload)} dataframes. Concatenating...")
df_for_bq_upload_concat = pd.concat(df_list_for_bq_upload)

In [ ]:
df_for_bq_upload_concat.head(50)

In [ ]:
dataset_ref = client.dataset(dataset_id)
table_ref = dataset_ref.table(output_table)

print(f"Loading group_mapping into bigquery...")

job_config = bigquery.LoadJobConfig(
    schema=[],
    write_disposition="WRITE_APPEND",
    schema_update_options=[bigquery.SchemaUpdateOption.ALLOW_FIELD_ADDITION]
)

job = client.load_table_from_dataframe(df_for_bq_upload_concat, table_ref, job_config=job_config)  
job.result()  # Wait for job completion